In [1]:
%run init.ipynb

matchzoo version 0.0.1
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]


In [2]:
preprocessor = mz.preprocessors.CDSSMPreprocessor(
    truncated_length_left=10,
    truncated_length_right=10
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 2118/2118 [00:00<00:00, 6624.48it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval => NgramLetter: 100%|██████████| 18841/18841 [00:05<00:00, 3257.14it/s]
Building Vocabulary from a datapack.: 100%|██████████| 1614998/1614998 [00:00<00:00, 2641067.94it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 7895.16it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 3919.75it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 808256.78it/s]
Processing text_left with chain_transform of NgramLetter => WordHashing: 100%|██████████| 2118/2118 [00:02<00:00, 769.57it/s]
Processing text_right with chain_tran

In [4]:
preprocessor.context

{'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fadf0a25f28>,
 'vocab_size': 9645,
 'embedding_input_dim': 9645}

In [5]:
padding_callback = mz.models.CDSSM.get_default_padding_callback()
fixed_padding_callback = mz.dataloader.callbacks.CDSSMPadding(
    fixed_length_left=10,
    fixed_length_right=100
)

In [6]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    callback=padding_callback
)

testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=fixed_padding_callback
)

In [7]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['strides'] = 1
model.params['padding'] = 0
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (1): Tanh()
    (2): Dropout(p=0.8)
    (3): AdaptiveMaxPool1d(output_size=1)
    (4): Squeeze()
    (5): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): Conv1d(9645, 64, kernel_size=(3,), stride=(1,))
    (1): Tanh()
    (2): Dropout(p=0.8)
    (3): AdaptiveMaxPool1d(output_size=1)
    (4): Squeeze()
    (5): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (out): Sequential(
    (0): Linear(in_features=1, out_features=1, bias=True)
  )
)
Trainable params:

In [8]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-102 Loss-0.860]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4285 - normalized_discounted_cumulative_gain@5(0.0): 0.4984 - mean_average_precision(0.0): 0.4613



[Iter-204 Loss-0.641]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4529 - normalized_discounted_cumulative_gain@5(0.0): 0.5247 - mean_average_precision(0.0): 0.4713



[Iter-306 Loss-0.540]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4489 - normalized_discounted_cumulative_gain@5(0.0): 0.5385 - mean_average_precision(0.0): 0.4868



[Iter-408 Loss-0.436]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.459 - normalized_discounted_cumulative_gain@5(0.0): 0.5412 - mean_average_precision(0.0): 0.4955



[Iter-510 Loss-0.354]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4757 - normalized_discounted_cumulative_gain@5(0.0): 0.5536 - mean_average_precision(0.0): 0.5044



[Iter-612 Loss-0.288]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4564 - normalized_discounted_cumulative_gain@5(0.0): 0.5392 - mean_average_precision(0.0): 0.4925



[Iter-714 Loss-0.241]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4734 - normalized_discounted_cumulative_gain@5(0.0): 0.5536 - mean_average_precision(0.0): 0.5043



[Iter-816 Loss-0.208]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4774 - normalized_discounted_cumulative_gain@5(0.0): 0.5542 - mean_average_precision(0.0): 0.5058



[Iter-918 Loss-0.185]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4699 - normalized_discounted_cumulative_gain@5(0.0): 0.5444 - mean_average_precision(0.0): 0.5001



[Iter-1020 Loss-0.176]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.4788 - normalized_discounted_cumulative_gain@5(0.0): 0.5436 - mean_average_precision(0.0): 0.505

Cost time: 3705.389808654785s
